# Building Floor Plan Viewer

View building floor plans with AHU coloring

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

from sbsim.smart_control.utils.scenario_generator import (
    load_building_config,
    generate_scenario_from_config,
    load_scenario_from_parts,
)
from sbsim.smart_control.utils.floor_generator import BSPFloorPlan, MultiFloorPlan, assign_rooms_to_ahus

In [ ]:
# List available building configs
buildings_dir = Path("buildings")
building_files = sorted(buildings_dir.glob("*.yaml"))

print("Available buildings:")
for i, f in enumerate(building_files):
    print(f"  {i}: {f.name}")

In [ ]:
# SELECT BUILDINGS TO VIEW (by index or name)
SELECTED = [0, 1, 2, 3]  # First 4 buildings
# Or by name:
# SELECTED = ["building_0001.yaml", "building_0002.yaml"]

In [ ]:
def get_building_paths(selected):
    """Convert selection to list of paths."""
    paths = []
    for s in selected:
        if isinstance(s, int):
            paths.append(building_files[s])
        else:
            paths.append(buildings_dir / s)
    return paths

selected_paths = get_building_paths(SELECTED)
print(f"Selected {len(selected_paths)} buildings:")
for p in selected_paths:
    print(f"  {p.name}")

In [ ]:
def generate_floor_plan(config_path):
    """Generate floor plan from config and return zone_map with AHU assignments."""
    import random
    
    config = load_building_config(str(config_path))
    fp = config.floor_plan
    
    # Set seed for reproducibility
    if config.seed is not None:
        random.seed(config.seed)
        np.random.seed(config.seed)
    
    # Create generator
    if fp.num_floors > 1:
        generator = MultiFloorPlan(
            width=fp.width, height=fp.height, num_floors=fp.num_floors,
            min_room_size=fp.min_room_size, max_room_size=fp.max_room_size,
            wall_thickness=fp.wall_thickness, door_width=fp.door_width,
            split_variance=fp.split_variance, building_shape=fp.building_shape,
            shape_ratio=fp.shape_ratio, composite_coverage=fp.composite_coverage,
        )
    else:
        generator = BSPFloorPlan(
            width=fp.width, height=fp.height,
            min_room_size=fp.min_room_size, max_room_size=fp.max_room_size,
            wall_thickness=fp.wall_thickness, door_width=fp.door_width,
            split_variance=fp.split_variance, building_shape=fp.building_shape,
            shape_ratio=fp.shape_ratio, composite_coverage=fp.composite_coverage,
        )
    
    generator.generate()
    zone_map = generator.export_zone_map(padding=5)
    num_rooms = generator.get_num_rooms()
    
    # Get AHU assignments
    num_ahus = config.ahu.num_ahus
    ahu_assignments = assign_rooms_to_ahus(num_rooms, num_ahus, seed=config.seed)
    
    return {
        'name': config_path.stem,
        'zone_map': zone_map,
        'num_rooms': num_rooms,
        'num_ahus': num_ahus,
        'ahu_assignments': ahu_assignments,
        'num_floors': fp.num_floors,
        'width': fp.width,
        'height': fp.height,
    }

In [ ]:
def create_ahu_colored_map(zone_map, ahu_assignments):
    """Create a colored map where each room is colored by its AHU."""
    import cv2
    
    # Get room labels using connected components
    binary = np.uint8(zone_map == 0)  # Interior space
    num_labels, labels = cv2.connectedComponents(binary, connectivity=4)
    
    # Create AHU map: -1 for walls/exterior, 0-N for AHU index
    ahu_map = np.full_like(zone_map, -1, dtype=np.int32)
    
    # Map each room label to its AHU
    for ahu_idx, rooms in enumerate(ahu_assignments):
        for room_id in rooms:
            # room_id is like 'zone_id_1', extract the number
            room_num = int(room_id.replace('zone_id_', ''))
            # Label room_num corresponds to connected component label room_num
            ahu_map[labels == room_num] = ahu_idx
    
    return ahu_map, num_labels - 1  # Subtract background

In [ ]:
def plot_building_grid(buildings_data, cols=4):
    """Plot buildings in a grid with AHU coloring."""
    n = len(buildings_data)
    rows = (n + cols - 1) // cols
    
    # AHU colors
    ahu_colors = [
        '#e41a1c',  # Red
        '#377eb8',  # Blue
        '#4daf4a',  # Green
        '#984ea3',  # Purple
        '#ff7f00',  # Orange
        '#ffff33',  # Yellow
        '#a65628',  # Brown
        '#f781bf',  # Pink
    ]
    
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
    if rows == 1 and cols == 1:
        axes = np.array([[axes]])
    elif rows == 1:
        axes = axes.reshape(1, -1)
    elif cols == 1:
        axes = axes.reshape(-1, 1)
    
    for idx, data in enumerate(buildings_data):
        row, col = idx // cols, idx % cols
        ax = axes[row, col]
        
        zone_map = data['zone_map']
        ahu_map, _ = create_ahu_colored_map(zone_map, data['ahu_assignments'])
        
        # Create colored image
        rgb = np.ones((*zone_map.shape, 3))  # White background
        
        # Color walls gray
        rgb[zone_map == 1] = [0.3, 0.3, 0.3]
        
        # Color exterior light gray
        rgb[zone_map == 2] = [0.9, 0.9, 0.9]
        
        # Color rooms by AHU
        for ahu_idx in range(data['num_ahus']):
            color = plt.cm.colors.to_rgb(ahu_colors[ahu_idx % len(ahu_colors)])
            mask = ahu_map == ahu_idx
            rgb[mask] = color
        
        ax.imshow(rgb)
        ax.set_title(f"{data['name']}\n{data['num_rooms']} rooms, {data['num_ahus']} AHUs\n{data['num_floors']} floor(s), {data['width']}x{data['height']}")
        ax.axis('off')
        
        # Add legend
        patches = [mpatches.Patch(color=ahu_colors[i % len(ahu_colors)], label=f'AHU {i+1}') 
                   for i in range(data['num_ahus'])]
        ax.legend(handles=patches, loc='upper right', fontsize=8)
    
    # Hide empty axes
    for idx in range(n, rows * cols):
        row, col = idx // cols, idx % cols
        axes[row, col].axis('off')
    
    plt.tight_layout()
    return fig

In [ ]:
# Generate floor plans for selected buildings
print("Generating floor plans...")
buildings_data = []
for path in selected_paths:
    print(f"  {path.name}...", end=" ", flush=True)
    data = generate_floor_plan(path)
    buildings_data.append(data)
    print(f"{data['num_rooms']} rooms, {data['num_ahus']} AHUs")
print("Done!")

In [ ]:
# Plot the grid
fig = plot_building_grid(buildings_data, cols=4)
plt.show()

In [ ]:
# View a single building larger
def show_building(data, figsize=(10, 10)):
    """Show a single building with AHU coloring."""
    ahu_colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628', '#f781bf']
    
    zone_map = data['zone_map']
    ahu_map, _ = create_ahu_colored_map(zone_map, data['ahu_assignments'])
    
    rgb = np.ones((*zone_map.shape, 3))
    rgb[zone_map == 1] = [0.3, 0.3, 0.3]  # Walls
    rgb[zone_map == 2] = [0.9, 0.9, 0.9]  # Exterior
    
    for ahu_idx in range(data['num_ahus']):
        color = plt.cm.colors.to_rgb(ahu_colors[ahu_idx % len(ahu_colors)])
        rgb[ahu_map == ahu_idx] = color
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.imshow(rgb)
    ax.set_title(f"{data['name']} - {data['num_rooms']} rooms, {data['num_ahus']} AHUs")
    ax.axis('off')
    
    patches = [mpatches.Patch(color=ahu_colors[i % len(ahu_colors)], label=f'AHU {i+1} ({len(data["ahu_assignments"][i])} rooms)') 
               for i in range(data['num_ahus'])]
    ax.legend(handles=patches, loc='upper right')
    plt.show()

# Example: show first building larger
# show_building(buildings_data[0])

In [ ]:
# Summary table
import pandas as pd

summary = pd.DataFrame([{
    'Building': d['name'],
    'Floors': d['num_floors'],
    'Size': f"{d['width']}x{d['height']}",
    'Rooms': d['num_rooms'],
    'AHUs': d['num_ahus'],
    'Rooms/AHU': f"{d['num_rooms']/d['num_ahus']:.1f}",
} for d in buildings_data])

display(summary)